# Load libraries

In [1]:
import modeling
import evaluation
from utils import data_loading
import matplotlib.pyplot as plt
import pickle
from pprint import pprint
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

# Load data

In [2]:
train_data = data_loading.load_train(['zaloai', 'mailong25', 'facebook'])
test_data = data_loading.load_test(['zaloai', 'mailong25', 'facebook'])
train_data

question  \
0     Quang Hải giành được chức vô địch U21 quốc gia...   
1     Quang Hải giành được chức vô địch U21 quốc gia...   
2     Quang Hải giành được chức vô địch U21 quốc gia...   
3     Quang Hải giành được chức vô địch U21 quốc gia...   
4                      Mỗi hiệp bóng đá kéo dài bao lâu   
...                                                 ...   
5490                      Ai đã xác định Vaginailis?   
5491  Bộ phận nào trong quá khứ có quyền ra quyết đị...   
5492                     Lễ trao giải diễn ra khi nào?    
5493                          Các mẫu được nhắm đến ai?   
5494                                DEF có nghĩa là gì?   

                                                   text  label  
0     Năm 2013 , Nguyễn Quang Hải giành chức vô địch...   True  
1     Sau chức vô địch U-21 quốc gia 2013 , Nguyễn Q...   True  
2     Anh bắt đầu gia nhập lò đào tạo trẻ Hà Nội T&T...  False  
3     Năm 2011 , Nguyễn Quang Hải mới 14 tuổi được g...  False  
4     Một trận đấu bóng đá thông thường có hai hiệp ...   True  
...                                                 ...    ...  
5490  Có khoảng 58 triệu ca nhiễm bệnh này trong năm...   True  
5491  Hiến pháp Hoa Kỳ phân chia quyền lực giữa hai ...   True  
5492  Lễ trao giải Oscar lần thứ 89, tổ chức bởi Việ...   True  
5493  Dòng  Nokia 3000 chủ yếu là điện thoại tầm tru...   True  
5494  Dung dịch xử lý khí thải động cơ diesel (viết ...   True  

[24900 rows x 3 columns]

In [3]:
X_train = train_data.drop(columns=['label'])
y_train = train_data['label']
X_test = test_data.drop(columns=['label'])
y_test = test_data['label']

# Train & Tune models

## Random Forest

In [4]:
model = modeling.get_model('random-forest')(vectorizer='tfidf')

In [5]:
print("Default Parameters ")
print('='*50)
pprint(model.get_classifier_params())
print('='*50)

Default Parameters 
{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': 10,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


In [6]:
grid_params  = {
    'max_features' : ['auto', 'sqrt', 'log2'],
    'max_depth':[10,20,30],
    'criterion':['gini', 'entropy']
    
}
print("Tuning Parameters")
print('='*50)
pprint(grid_params)
print('='*50)

Tuning Parameters
{'criterion': ['gini', 'entropy'],
 'max_depth': [10, 20, 30],
 'max_features': ['auto', 'sqrt', 'log2']}


In [7]:
print('Best Parameters')
print('='*50)
print(model.grid_search(X_train, y_train, grid_params, scoring='f1_macro'))
print('='*50)

Best Parameters
Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-2)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=-2)]: Done  36 tasks      | elapsed:   44.3s
[Parallel(n_jobs=-2)]: Done  54 out of  54 | elapsed:  1.3min finished


{'criterion': 'entropy', 'max_depth': 10, 'max_features': 'sqrt'}


In [8]:
model = modeling.get_model('random-forest')(vectorizer='tfidf',max_depth = 10, max_features='sqrt', criterion = 'entropy')

In [9]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [10]:
evaluation.get_f1_score(y_test, y_pred)

0.6501356824434761